# Load Dependencies

In [1]:
import numpy as np
import pandas as pd; pd.options.display.max_columns = 200
import geopandas as gpd
import linref as lr

# Load Data

In [2]:
fp = '/mnt/h/H0000/00CMH.00251.00_S_ODOT HSIP Safety Services/02C Python Training/Example Data/franklin_county_sample_data.gpkg'
gpd.list_layers(fp)

/home/tshihadah/code/python-training/venv/lib/python3.12/site-packages/pyogrio/core.py:130: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured MultiLineString' is converted to 'MultiLineString'
  return ogr_list_layers(get_vsi_path_or_buffer(path_or_buffer))


,name,geometry_type
0,Crashes_2019_2023,Point
1,road_inventory_franklin_county — road_inventory,MultiLineString
2,layer_styles,None


In [31]:
# Point this to the location of the Geopackage file
fp = '/mnt/h/H0000/00CMH.00251.00_S_ODOT HSIP Safety Services/02C Python Training/Example Data/franklin_county_sample_data.gpkg'

# Load crash data
crashes = gpd.read_file(fp, layer='Crashes_2019_2023').to_crs('EPSG:3857')

# Load road data
roadways = gpd.read_file(fp, layer='road_inventory_franklin_county — road_inventory').to_crs('EPSG:3857')

/home/tshihadah/code/python-training/venv/lib/python3.12/site-packages/pyogrio/raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured MultiLineString' is converted to 'MultiLineString'
  return ogr_read(


In [32]:
roadways_select = roadways[roadways.FUNCTION_C.isin([3, 4, 5, 6])]
roadways_select.shape

(5110, 103)

In [37]:
spatial_matches = crashes.sjoin_nearest(roadways_select[['geometry']], 'inner', max_distance=100, exclusive=False)
crashes_select = crashes[(crashes.CRASH_SEVERITY_CD.isin([1, 2, 3, 4])) & crashes.NLFID.isin(roadways_select.NLF_ID) & (crashes.OBJECTID.isin(spatial_matches.OBJECTID))]
crashes_select.shape

(25872, 150)

In [46]:
fp = '/mnt/h/H0000/00CMH.00251.00_S_ODOT HSIP Safety Services/02C Python Training/Example Data/franklin_county_training_data.gpkg'
crashes_select.to_file(fp, layer='crashes', driver='GPKG')
roadways_select.to_file(fp, layer='roadways', driver='GPKG')

# Crash Data Cleanup

## Decoding and Binning

Data dictionary available here: https://www.dot.state.oh.us/Divisions/Planning/LocalPrograms/LTAP/Documents/Crash_Data_Dictionary.pdf?ID=1229/1000

# Roadway Data Cleanup

# Linear Referencing